In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score

In [2]:
open("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey_ML.csv")
df = pd.read_csv("/Users/olivianystrom/Desktop/Springboard/CA_Birds_of_Prey/Birds_of_Prey_ML.csv")


In [3]:
print(df.shape)
print(df.head())

(4239061, 29)
   Unnamed: 0        COMMON NAME           SCIENTIFIC NAME OBSERVATION DATE  \
0           0         Bald Eagle  Haliaeetus leucocephalus       1999-01-18   
1           1      Cooper's Hawk        Accipiter cooperii       1996-10-13   
2           2    Red-tailed Hawk         Buteo jamaicensis       1997-08-22   
3           3   American Kestrel          Falco sparverius       1995-09-11   
4           4  White-tailed Kite           Elanus leucurus       1994-10-15   

                     Size  Brown  White  Gray  Orange  Black  ... robin-sized  \
0   goose-sized or larger      1      1     0       0      0  ...           0   
1              crow-sized      0      0     1       1      0  ...           0   
2  between crow and goose      1      1     0       1      0  ...           0   
3             robin-sized      0      0     1       1      1  ...           1   
4  between crow and goose      0      1     1       0      1  ...           0   

  between robin and crow

In [4]:
# Split into test and train data

X_train, X_test, y_train, y_test = train_test_split(df[['Brown', 'White',
       'Gray', 'Orange', 'Black','between sparrow and robin', 'robin-sized',
       'between robin and crow', 'crow-sized', 'between crow and goose', 'Lat_normalized', 
       'Long_normalized']], df[['Hawk', 'Falcon', 'Kite', 'Eagle','Owl']], 
       test_size=0.33, random_state=42)

X_train_name, X_test_name, y_train_name, y_test_name = train_test_split(df[['Brown', 'White',
       'Gray', 'Orange', 'Black','between sparrow and robin', 'robin-sized',
       'between robin and crow', 'crow-sized', 'between crow and goose', 'Lat_normalized', 
       'Long_normalized']], df['COMMON NAME_cat'], 
       test_size=0.33, random_state=42)



In [5]:
X_train.shape
# y_train.shape

(2840170, 12)

In [11]:
X_train_small = X_train.head(500000)
y_train_small = y_train.head(500000)

X_train_name_small = X_train_name.head(500000)
y_train_name_small = y_train_name.head(500000)

In [5]:
#starting with the random forest model, only classifying into Category (hawk/kestrel/falcon/etc)

clf = RandomForestClassifier(n_estimators=300, random_state = 1,n_jobs=-1)
model_res = clf.fit(X_train, y_train)
y_pred = model_res.predict(X_test)
y_pred_prob = model_res.predict_proba(X_test)

ac = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='weighted')

print('Random Forest: Accuracy=%.3f' % (ac))

print('Random Forest: f1-score=%.3f' % (f1))

Random Forest: Accuracy=0.951
Random Forest: f1-score=0.941


In [6]:
#Still random forest, this time predicting species 

clf = RandomForestClassifier(n_estimators=300, random_state = 1,n_jobs=-1)
model_res_name = clf.fit(X_train_name, y_train_name)
y_pred_name = model_res_name.predict(X_test_name)
y_pred_prob_name = model_res_name.predict_proba(X_test_name)

ac_name = accuracy_score(y_test_name, y_pred_name)

f1_name = f1_score(y_test_name, y_pred_name, average='weighted')

print('Random Forest: Accuracy=%.3f' % (ac_name))

print('Random Forest: f1-score=%.3f' % (f1_name))

#Species result is actually fairly accurate, will continue to Classify by species moving forward

Random Forest: Accuracy=0.824
Random Forest: f1-score=0.815


In [12]:
#Using XGBoost Classifier, classifying by species and only using 500,000 lines of the train data because the code 
#took a long time to run. XGBoost is run with the full dataset in the next cell

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_name_small, y_train_name_small)
    
y_pred_name = xgb_model.predict(X_test_name)
# y_pred_name = pd.DataFrame(y_pred_name)

ac_name = accuracy_score(y_test_name, y_pred_name)

f1_name = f1_score(y_test_name, y_pred_name, average='weighted')

print('XGBoost: Accuracy=%.3f' % (ac_name))

print('XGBoost: f1-score=%.3f' % (f1_name))

#Results are not as good as the Random Forest model but still comparable

/Users/olivianystrom/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:17:01] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost: Accuracy=0.817
XGBoost: f1-score=0.797


In [13]:
#XGBoost Classifier run to predict species with the full dataset

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_name, y_train_name)
    
y_pred_name = xgb_model.predict(X_test_name)
# y_pred_name = pd.DataFrame(y_pred_name)

ac_name = accuracy_score(y_test_name, y_pred_name)

f1_name = f1_score(y_test_name, y_pred_name, average='weighted')

print('XGBoost: Accuracy=%.3f' % (ac_name))

print('XGBoost: f1-score=%.3f' % (f1_name))

#Accuracy and fl-score are better here, but not by much, and while comparable with the Random Forest model
#The Random Forest model ran much more quickly. If we were to refine this, I would use the RF model and consider
#chopping down the train code since we can significantly reduce the runtime of the code without much impact on 
#Accuracy

/Users/olivianystrom/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:21:21] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost: Accuracy=0.820
XGBoost: f1-score=0.799
